In [1]:
from newspaper import Article
from newspaper.outputformatters import OutputFormatter
from newspaper.configuration import Configuration

from urllib.parse import urlsplit


import re

In [2]:
conf = Configuration()

In [3]:
conf.keep_article_html = True

In [4]:
conf.keep_article_html

True

In [5]:
unz_url = "http://www.unz.com/pgiraldi/the-international-zionist-conspiracy/"
sf_url = "https://southfront.org/philip-m-giraldi-the-international-zionist-conspiracy/"
unz_article = Article(unz_url, keep_article_html=True)
sf_article = Article(sf_url, keep_article_html=True)

In [6]:
test_article1_l = ["https://oneworld.press/?module=articles&action=view&id=1257"]
test_article2_l = ["https://southfront.org/how-and-why-iran-shot-down-ukrainian-boeing/"]
test_article3_l = ["https://www.bbc.com/news/world-middle-east-51042326"]

In [7]:
link_list_en = list(set(["https://www.strategic-culture.org/news/2020/01/21/poles-and-jews-before-wwii/",
             "https://orientalreview.org/2020/01/20/heres-why-iran-stopped-covering-up-its-role-in-the-uia-752-tragedy/",
             "https://journal-neo.org/2020/01/20/pompeo-new-us-policy-authorized-drones-strikes-on-russian-chinese-leaders/",
             "https://www.rt.com/news/478734-ukraine-trident-uk-terrorism/",
             "https://southfront.org/western-doublespeak-towards-ukrainian-conflict/",
             "https://www.rt.com/news/478610-russia-fight-history-distortion-putin/",
             "https://www.rt.com/shows/news/478504-rtnews-january-17-17msk/",
             "https://sputniknews.com/military/202001171078061694-declassified-wwii-americans-brits-killed-polish-death-camp/",
             "https://www.fort-russ.com/2020/01/ishchenko-jews-and-the-holocaust-as-ukraines-way-of-life/",
             "https://www.fort-russ.com/2020/01/texas-iranian-flight-crash-facts-not-adding-up/",
             "https://www.theyeshivaworld.com/news/featured/1821774/pogrom-in-uman-over-shabbos-ukrainian-thugs-attack-jews-4-evacuated-to-hospitals.html",
             "https://www.rt.com/shows/news/477904-rtnews-january-10-17msk/",
             "https://sputniknews.com/europe/202001091077992890-polish-lawmakers-adopt-resolution-equally-blaming-germany-soviet-union-for-start-of-wwii/",
             "https://www.rt.com/shows/news/477622-rtnews-january-07-17msk/",
             "https://oneworld.press/?module=articles&action=view&id=1241",
             "https://orientalreview.org/2020/01/07/latest-mh17-documentary-shares-some-shocking-truths/",
             "https://www.strategic-culture.org/news/2019/12/28/western-campaign-against-orthodox-church-turns-to-montenegro/",
             "https://russia-insider.com/en/western-campaign-against-orthodox-church-turns-montenegro/ri28125",
             "https://sputniknews.com/europe/201912191077630056-Putin-Says-Russia-Will-Do-Everything-to-Support-Sputniks-Work-Abroad/",
             "https://www.rt.com/news/476141-nato-drills-war-scenario/",
             "https://web.archive.org/web/20200116041701/http://webcache.googleusercontent.com/search?q=cache:nxzKIeN-cZMJ:thejewishlink.com/pogrom-in-uman-over-shabbos-ukrainian-thugs-attack-jews-4-evacuated-to-hospitals/+&cd=3&hl=ru&ct=clnk&gl=ua"
               ]))

link_list_ru = set(["https://news-front.info/2019/12/17/ukraina-mozhet-pogruzitsya-v-gomodiktaturu/",
                "https://sputnik-georgia.ru/video/20200119/247517930/V-Moskve-dali-salyut-v-chest-75-y-godovschiny-osvobozhdeniya-Varshavy.html",
                "https://www.rbc.ru/rbcfreenews/5e20d5789a7947857c0c8176"
               ])

In [8]:
def read_url(url):
    art = Article(url, keep_article_html=True)
    art.download()
    art.parse()
    return art

def base_url_from_article(article):
    url = article.url
    return get_base_url(url)

def get_base_url(url):
    split_url = urlsplit(url)
    base_url = split_url.netloc
    return base_url

def read_link_list(link_list):
    articles = []
    for url in link_list:
        articles.append( read_url(url) )
    return articles

In [9]:
articles = read_link_list(link_list_en)

In [10]:
articles

In [87]:
base_urls = set([base_url_from_article(article) for article in articles])

In [75]:
articles = []
base_urls = set()
for url in link_list_en:
    art = Article(url, keep_article_html=True)
    art.download()
    art.parse()
    articles.append(art)
    split_url = urlsplit(url)
    base_urls.add(split_url.netloc)

In [88]:
base_urls.remove('web.archive.org')

In [89]:
base_urls

{'journal-neo.org',
 'oneworld.press',
 'orientalreview.org',
 'russia-insider.com',
 'southfront.org',
 'sputniknews.com',
 'www.fort-russ.com',
 'www.rt.com',
 'www.strategic-culture.org',
 'www.theyeshivaworld.com'}

In [78]:
len(articles)

21

In [12]:
articles[0].authors

['Cechas Vodobenikov', 'Am Hants']

In [13]:
articles[0].url

'https://southfront.org/western-doublespeak-towards-ukrainian-conflict/'

In [91]:
articles_text = [article.text for article in articles]

In [121]:
test_articles1 = read_link_list(test_article3_l)

test_articles1_text = [article.text for article in test_articles1]

In [25]:
articles_ru = []
for url in link_list_ru:
    art = Article(url, keep_article_html=True)
    art.download()
    art.parse()
    articles_ru.append(art)

In [26]:
articles_ru

In [33]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [92]:
tfidf = TfidfVectorizer()

In [93]:
tfidf_vec = tfidf.fit_transform(articles_text)

In [122]:
tfidf_test = tfidf.transform(test_articles1_text)

In [123]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(tfidf_test, tfidf_vec).flatten()
cosine_similarities

array([0.43827705, 0.38992142, 0.36433358, 0.49270493, 0.24966082,
       0.38588573, 0.36617791, 0.50057822, 0.49283494, 0.2998851 ,
       0.49270493, 0.40637352, 0.5409709 , 0.27275934, 0.41394773,
       0.34205507, 0.43603442, 0.50655837, 0.5051394 , 0.50924971,
       0.4955035 ])

In [124]:
related_docs_indices = cosine_similarities.argsort()[:-5:-1]
related_docs_indices

array([12, 19, 17, 18])

In [118]:
link_list_en[19]

'https://southfront.org/western-doublespeak-towards-ukrainian-conflict/'

In [125]:
cosine_similarities[related_docs_indices]

array([0.5409709 , 0.50924971, 0.50655837, 0.5051394 ])

In [113]:
test_article1_l

['https://oneworld.press/?module=articles&action=view&id=1257']

In [137]:
import torch

from flair.data import Sentence
from flair.embeddings import FlairEmbeddings, DocumentPoolEmbeddings, WordEmbeddings

In [177]:
# first, declare how you want to embed
embeddings = DocumentPoolEmbeddings(
    [ FlairEmbeddings('news-forward'), FlairEmbeddings('news-backward')])

In [178]:
# your query
query = Sentence(test_articles1_text[0][0:500])

In [179]:
# some texts
paragraph_1 = Sentence(articles_text[0][0:500])
paragraph_2 = Sentence(articles_text[1][0:500])

In [180]:
# embed everything
embeddings.embed([ paragraph_1, paragraph_2])

In [181]:
# use cosine distance
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [182]:
embeddings.embed([query])

In [183]:
# get similarity between embeddings of query and paragraph 1
similarity_to_paragraph_1 = cos(query.embedding, paragraph_1.embedding)
print(similarity_to_paragraph_1)

tensor(0.6222, grad_fn=<DivBackward0>)


In [184]:
similarity_to_paragraph_2 = cos(query.embedding, paragraph_2.embedding)
print(similarity_to_paragraph_2)

tensor(0.6900, grad_fn=<DivBackward0>)


In [185]:
test_articles1_text[0][0:500]

"Media playback is unsupported on your device Media caption Video from the scene shows the aftermath of the crash\n\nIran has said it will not hand over black box flight recorders recovered from a plane that crashed with 176 people on board to manufacturer Boeing or to the US.\n\nThe Ukrainian Boeing 737-800 went down just minutes after taking off from Tehran's airport, leaving no survivors.\n\nUnder global aviation rules, Iran has the right to lead the investigation.\n\nBut manufacturers are typically i"

In [186]:
articles_text[0][0:500]

'Moscow is to create the most extensive collection of WWII documents, open to all persons anywhere, to once and for all “shut the filthy mouth” of those seeking to rewrite history for short-term gains, the Russian president said.\n\nAny person, Russian or non-national, will be able to access the archive, including through a website resource, and the ultimate goal is to debunk any disinformation about the most devastating conflict in human history, President Vladimir Putin pledged, during a meeting '

In [161]:
articles_text[1][0:500]

'The increased intensity and scale of US-led NATO military exercises in the Baltics and Black Sea suggest the alliance is preparing for a major military conflict, said General Valery Gerasimov, chief of the Russian General Staff.\n\nNATO’s ramping up of large-scale war games in the Baltics, Poland and the Black Sea region show that it is gearing up for a major confrontation, Gerasimov said on Thursday, briefing foreign military attaches in Moscow.\n\n“The scenario [of the drills] point that NATO is p'

In [8]:
unz_article.download()
sf_article.download()

In [9]:
unz_article.parse()
sf_article.parse()

In [10]:
unz_article.authors

['Philip Giraldi', 'Gilad Atzmon', 'Ron Unz', 'Laurent Guyénot']

In [11]:
sf_article.authors

['Karen Bartlett',
 'Concrete Mike',
 'Ivan Freely',
 'Tommy Jensen',
 'Sam Navarro']

In [12]:
unz_article.publish_date

In [13]:
sf_article.publish_date

datetime.datetime(2019, 12, 3, 13, 26, 4, tzinfo=tzutc())

In [14]:
unz_article.text[0:1000]

'A recent article by Philip Weiss on the Mondoweiss website lays out an argument that most liberal Jews, like Weiss, are hesitant to support, namely that Jewish power, and more to the point its money, as exercised through the so-called Israel Lobby in the United States and elsewhere, has been the principal enabling force behind the international pariah that the state of Israel has become.\n\nWeiss notes how “most observers accept the antisemitism redlines echoed lately by Bernie Sanders: you are not to speak of an outsize Jewish role in politics. So few write about the Israel lobby, though they know it to be a significant force…” In other words, Sanders, liberal to the core and ostensibly supportive of Palestinian rights, draws a line that forbids any real discussion of Jewish power in the United States even though everyone who has not been asleep is more than aware of just how powerful American Jews, and by extension Israel, are.\n\nWeiss details how the vast sums of money raised by b

In [15]:
sf_article.text[0:1000]

'Donate\n\nWritten by Philip M. Giraldi; Originally appeared at The Unz Review\n\nA recent article by Philip Weiss on the Mondoweiss website lays out an argument that most liberal Jews, like Weiss, are hesitant to support, namely that Jewish power, and more to the point its money, as exercised through the so-called Israel Lobby in the United States and elsewhere, has been the principal enabling force behind the international pariah that the state of Israel has become.\n\nWeiss notes how “most observers accept the antisemitism redlines echoed lately by Bernie Sanders: you are not to speak of an outsize Jewish role in politics. So few write about the Israel lobby, though they know it to be a significant force…” In other words, Sanders, liberal to the core and ostensibly supportive of Palestinian rights, draws a line that forbids any real discussion of Jewish power in the United States even though everyone who has not been asleep is more than aware of just how powerful American Jews, and 

In [133]:
unz_article.canonical_link

'http://www.unz.com/pgiraldi/the-international-zionist-conspiracy/'

In [134]:
sf_article.canonical_link

'https://southfront.org/philip-m-giraldi-the-international-zionist-conspiracy/'

In [135]:
article.link_hash

'f41ffef6c9510fec4cc814e189b82f03.1575814617.2237463'

In [136]:
unz_article.nlp()
sf_article.nlp()

In [137]:
unz_article.keywords

['party',
 'israel',
 'states',
 'zionist',
 'jews',
 'palestinian',
 'money',
 'jewish',
 'power',
 'support',
 'united',
 'conspiracy',
 'international']

In [138]:
sf_article.keywords

['party',
 'israel',
 'states',
 'zionist',
 'philip',
 'jews',
 'm',
 'palestinian',
 'money',
 'jewish',
 'power',
 'support',
 'united',
 'conspiracy',
 'international',
 'giraldi']

In [139]:
unz_article.summary

'1, Israeli settlement/colonialism (or in Zionist terms, the effort to liberate European Jewry from persecution by establishing a Jewish homeland in historical Palestine).\nWithout the blind support of the United States, Israel would have made a deal a long time ago.\nWithout U.S. support, Israel would have been internationally isolated and would have grabbed the deal.\nAnd the Israel conspiracy might well be regarded as international.\nBritain is already in some senses Zionist controlled territory, even more so than the United States.'

In [141]:
sf_article.summary

'He describes how president after president has backed down versus Israel when confronted by Jewish power and observes that “This is not just a domestic political question, it’s a foreign policy problem.\n1, Israeli settlement/colonialism (or in Zionist terms, the effort to liberate European Jewry from persecution by establishing a Jewish homeland in historical Palestine).\nWithout the blind support of the United States, Israel would have made a deal a long time ago.\nWithout U.S. support, Israel would have been internationally isolated and would have grabbed the deal.\nBritain is already in some senses Zionist controlled territory, even more so than the United States.'

In [142]:
unz_article.meta_data

defaultdict(dict,
            {'viewport': 'width=device-width, initial-scale=1',
             'og': {'site_name': 'The Unz Review',
              'locale': 'en_us',
              'type': 'article',
              'title': 'The International Zionist Conspiracy',
              'url': 'http://www.unz.com/pgiraldi/the-international-zionist-conspiracy/',
              'description': 'It poisons everything it touches',
              'image': 'http://www.unz.com/wp-content/uploads/2019/12/Kraft-and-Abramovich.jpg'},
             'fb': {'app_id': 158057877736603}})

In [143]:
sf_article.meta_data

defaultdict(dict,
            {'viewport': 'width=device-width, initial-scale=1, shrink-to-fit=no',
             'robots': 'max-snippet:-1, max-image-preview:large, max-video-preview:-1',
             'og': {'locale': 'en_GB',
              'type': 'article',
              'title': 'Philip M. Giraldi: "The International Zionist Conspiracy"',
              'description': 'Written by Philip M. Giraldi; Originally appeared at The Unz Review A\xa0recent article\xa0by Philip Weiss on the\xa0Mondoweiss\xa0website lays out an...',
              'url': 'https://southfront.org/philip-m-giraldi-the-international-zionist-conspiracy/',
              'image': {'identifier': 'https://southfront.org/wp-content/uploads/2019/12/1-16.jpg',
               'secure_url': 'https://southfront.org/wp-content/uploads/2019/12/1-16.jpg',
               'width': 640,
               'height': 400}},
             'article': {'publisher': 'https://www.facebook.com/SouthFrontEnTwo',
              'section': "Editor's

In [144]:
unz_article.meta_lang

''

In [145]:
sf_article.meta_lang

'en'

In [156]:
sf_article.fetch_images()

In [158]:
sf_article.images

{'https://informer.yandex.ru/informer/35327630/3_1_FFFFFFFF_EFEFEFFF_0_pageviews',
 'https://mc.yandex.ru/watch/35327630',
 'https://southfront.org/wp-content/themes/wt_tera/images/AM.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/DE.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/100-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/20-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/200-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/50-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/500-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/another-amount-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/logo.png',
 'https://southfront.org/wp-content/uploads/2015/12/DONATE-EUR.png',
 'https://southfront.org/wp-content/uploads/2016/01/DONATE-USD.png',
 'https://southfront.org/wp-content/uploads/2016/07/o

In [162]:
print(sf_article.top_image)

https://southfront.org/wp-content/uploads/2019/12/1-16.jpg


In [161]:
sf_article.imgs

{'https://informer.yandex.ru/informer/35327630/3_1_FFFFFFFF_EFEFEFFF_0_pageviews',
 'https://mc.yandex.ru/watch/35327630',
 'https://southfront.org/wp-content/themes/wt_tera/images/AM.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/DE.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/100-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/20-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/200-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/50-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/500-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/donate/another-amount-min.png',
 'https://southfront.org/wp-content/themes/wt_tera/images/logo.png',
 'https://southfront.org/wp-content/uploads/2015/12/DONATE-EUR.png',
 'https://southfront.org/wp-content/uploads/2016/01/DONATE-USD.png',
 'https://southfront.org/wp-content/uploads/2016/07/o

In [149]:
unz_html = unz_article.article_html
sf_html = sf_article.article_html

In [150]:
sf_html

'<div><p class="no-break">         Donate</p><p>Written by <strong>Philip M. Giraldi</strong>; Originally appeared at <a href="http://www.unz.com/pgiraldi/the-international-zionist-conspiracy/" target="_blank" rel="noopener noreferrer"><strong>The Unz Review</strong></a></p><p id="p_1_1" class="container">A&#160;<a title="https://mondoweiss.net/2019/11/the-root-cause-of-the-conflict-is-the-israel-lobby/" href="https://mondoweiss.net/2019/11/the-root-cause-of-the-conflict-is-the-israel-lobby/">recent article</a>&#160;by Philip Weiss on the&#160;Mondoweiss&#160;website lays out an argument that most liberal Jews, like Weiss, are hesitant to support, namely that Jewish power, and more to the point its money, as exercised through the so-called Israel Lobby in the United States and elsewhere, has been the principal enabling force behind the international pariah that the state of Israel has become.</p><p><a href="https://southfront.org/wp-content/uploads/2019/12/1-16.jpg" target="_blank" rel

In [153]:
re.findall(r"(?:<a .*?href=\")(.*?)(?:\")", unz_html)

['https://mondoweiss.net/2019/11/the-root-cause-of-the-conflict-is-the-israel-lobby/',
 'https://foreignpolicy.com/2016/06/29/inside-the-democratic-partys-showdown-over-israel-palestine/',
 'http://www.gazettes.com/news/politics/democrats-make-platform-decisions-on-final-day-in-long-beach/article_b0264c36-09b1-11ea-bf76-13b9e830caca.html?fbclid=IwAR3RQgEiItMGrfGL-uawTO5UKedhq_Q921yl0yzY_-_sdAynL5pmDaOn9CQ',
 'https://www.thetimes.co.uk/edition/news/ephraim-mirvis-what-will-become-of-jews-in-britain-if-labour-forms-the-next-government-ghpsdbljk',
 'https://mondoweiss.net/2019/11/robert-kraft-launches-million-dollar-competition-to-combat-the-bds-movement/',
 'https://jewishnews.timesofisrael.com/chelsea-ownera-pledges-3-9m-for-new-robert-kraft-antisemitism-foundation/',
 'https://en.wikipedia.org/wiki/Roman_Abramovich',
 'https://en.wikipedia.org/wiki/Robert_Kraft',
 'https://www.ynetnews.com/article/HJzOUY6nS',
 'http://www.itamaraty.gov.br/en/press-releases/20236-joint-declaration-on-t

In [154]:
re.findall(r"(?:<a .*?href=\")(.*?)(?:\")", sf_html)

['http://www.unz.com/pgiraldi/the-international-zionist-conspiracy/',
 'https://mondoweiss.net/2019/11/the-root-cause-of-the-conflict-is-the-israel-lobby/',
 'https://southfront.org/wp-content/uploads/2019/12/1-16.jpg',
 'https://foreignpolicy.com/2016/06/29/inside-the-democratic-partys-showdown-over-israel-palestine/',
 'http://www.gazettes.com/news/politics/democrats-make-platform-decisions-on-final-day-in-long-beach/article_b0264c36-09b1-11ea-bf76-13b9e830caca.html?fbclid=IwAR3RQgEiItMGrfGL-uawTO5UKedhq_Q921yl0yzY_-_sdAynL5pmDaOn9CQ',
 'https://www.thetimes.co.uk/edition/news/ephraim-mirvis-what-will-become-of-jews-in-britain-if-labour-forms-the-next-government-ghpsdbljk',
 'https://mondoweiss.net/2019/11/robert-kraft-launches-million-dollar-competition-to-combat-the-bds-movement/',
 'https://jewishnews.timesofisrael.com/chelsea-ownera-pledges-3-9m-for-new-robert-kraft-antisemitism-foundation/',
 'https://en.wikipedia.org/wiki/Roman_Abramovich',
 'https://en.wikipedia.org/wiki/Rober